In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
from peft import PeftModel
import re


# ----------------------------
# CONFIG
# ----------------------------
BASE_MODEL = "google-t5/t5-small"
LORA_PATH = r"C:\Users\HP\Desktop\NMAP_AI\backend\Agents\Agent_medium\T5\T5_qlora-nmap"   # folder you saved

# ----------------------------
# Quantization (same as training)
# ----------------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# ----------------------------
# Load base model
# ----------------------------
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto"
)

# ----------------------------
# Load LoRA adapters
# ----------------------------
model = PeftModel.from_pretrained(base_model, LORA_PATH)
model.eval()

# ----------------------------
# Load tokenizer
# ----------------------------
tokenizer = AutoTokenizer.from_pretrained(LORA_PATH)

# ----------------------------
# MEDIUM COMMAND TESTS
# ----------------------------
medium_prompts = [
    "Scan all ports on 192.168.1.20",
    "Scan all ports and detect services on 192.168.1.20",
    "Scan all TCP ports using SYN scan on 192.168.1.20",
    "Scan top ports with version detection on 192.168.1.50",
    "Run default scripts on 192.168.1.10",
    "Run default and vuln scripts on 192.168.1.10",
    "Scan a subnet for live hosts using ping scan",
    "Perform OS detection on 192.168.1.100"
]

# ----------------------------
# Generate & validate
# ----------------------------
print("\n" + "="*60)
print("🧪 MEDIUM COMMAND TESTS")
print("="*60 + "\n")

for prompt in medium_prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=60,
            do_sample=False,
            num_beams=1
        )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    result = result.split("\n")[0].strip()

    # Simple validation (medium-agent safe)
    is_valid = bool(re.match(r"^nmap\s+", result))

    print(f"📝 Prompt : {prompt}")
    print(f"✅ Command : {result}")
    print(f"🔍 Valid  : {'YES' if is_valid else 'NO'}\n")


🧪 MEDIUM COMMAND TESTS

📝 Prompt : Scan all ports on 192.168.1.20
✅ Command : nmap -p- 192.168.1.20
🔍 Valid  : YES

📝 Prompt : Scan all ports and detect services on 192.168.1.20
✅ Command : nmap -p- -sV 192.168.1.20
🔍 Valid  : YES

📝 Prompt : Scan all TCP ports using SYN scan on 192.168.1.20
✅ Command : nmap -p- -sS 192.168.1.20
🔍 Valid  : YES

📝 Prompt : Scan top ports with version detection on 192.168.1.50
✅ Command : nmap -p- -sV 192.168.1.50
🔍 Valid  : YES

📝 Prompt : Run default scripts on 192.168.1.10
✅ Command : nmap -p 80 --script default 192.168.1.10
🔍 Valid  : YES

📝 Prompt : Run default and vuln scripts on 192.168.1.10
✅ Command : nmap -p- 192.168.1.10
🔍 Valid  : YES

📝 Prompt : Scan a subnet for live hosts using ping scan
✅ Command : nmap -sn --subnet -sn -sV --host-hosts.pvv.txt target
🔍 Valid  : YES

📝 Prompt : Perform OS detection on 192.168.1.100
✅ Command : nmap -O 192.168.1.100
🔍 Valid  : YES



In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
from peft import PeftModel
import re


# ----------------------------
# CONFIG
# ----------------------------
BASE_MODEL = "google-t5/t5-small"
LORA_PATH = r"C:\Users\HP\Desktop\NMAP_AI\backend\Agents\Agent_medium\T5\T5_qlora-nmap"

# ----------------------------
# Quantization (same as training)
# ----------------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# ----------------------------
# Load base model
# ----------------------------
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto"
)

# ----------------------------
# Load LoRA adapters
# ----------------------------
model = PeftModel.from_pretrained(base_model, LORA_PATH)
model.eval()

# ----------------------------
# Load tokenizer
# ----------------------------
tokenizer = AutoTokenizer.from_pretrained(LORA_PATH)

# ----------------------------
# VALIDATION + RULES
# ----------------------------

VALID_NMAP = re.compile(r"^nmap\s+.+")
FORBIDDEN_TOKENS = ["--subnet", "target", "hosts.pvv", "hosts.txt"]

def apply_rules(prompt: str, cmd: str):
    """
    Light rule-based corrections for MEDIUM agent
    """
    original_cmd = cmd
    p = prompt.lower()

    # Rule 1: top ports ≠ all ports
    if "top ports" in p and "-p-" in cmd:
        cmd = cmd.replace("-p-", "").strip()

    # Rule 2: default + vuln scripts
    if "default and vuln" in p:
        cmd = re.sub(r"--script\s+\S+", "--script default,vuln", cmd)
        if "--script" not in cmd:
            cmd = f"nmap --script default,vuln {cmd.replace('nmap', '').strip()}"

    # Rule 3: default scripts only
    if "default scripts" in p and "vuln" not in p:
        if "--script" not in cmd:
            cmd = f"nmap --script default {cmd.replace('nmap', '').strip()}"

    # Rule 4: ping scan → only -sn
    if "ping scan" in p:
        cmd = re.sub(r"-sV|-O|-p-|--script\s+\S+", "", cmd).strip()
        if "-sn" not in cmd:
            cmd = f"nmap -sn {cmd.replace('nmap', '').strip()}"

    corrected = (cmd != original_cmd)
    return cmd.strip(), corrected


def is_valid_nmap(cmd: str) -> bool:
    if not VALID_NMAP.match(cmd):
        return False
    for bad in FORBIDDEN_TOKENS:
        if bad in cmd:
            return False
    return True


# ----------------------------
# MEDIUM COMMAND TESTS
# ----------------------------
medium_prompts = [
    "Scan all ports on 192.168.1.20",
    "Scan all ports and detect services on 192.168.1.20",
    "Scan all TCP ports using SYN scan on 192.168.1.20",
    "Scan top ports with version detection on 192.168.1.50",
    "Run default scripts on 192.168.1.10",
    "Run default and vuln scripts on 192.168.1.10",
    "Scan a subnet for live hosts using ping scan on 192.168.1.0/24",
    "Perform OS detection on 192.168.1.100"
]

# ----------------------------
# RUN TESTS
# ----------------------------
print("\n" + "=" * 60)
print("🧪 MEDIUM COMMAND TESTS (WITH RULES)")
print("=" * 60 + "\n")

for prompt in medium_prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=60,
            do_sample=False,
            num_beams=1
        )

    raw_cmd = tokenizer.decode(outputs[0], skip_special_tokens=True)
    raw_cmd = raw_cmd.split("\n")[0].strip()

    final_cmd, corrected = apply_rules(prompt, raw_cmd)
    valid = is_valid_nmap(final_cmd)

    print(f"📝 Prompt : {prompt}")
    print(f"🤖 Raw    : {raw_cmd}")
    if corrected:
        print(f"🛠 Fixed  : {final_cmd}")
    else:
        print(f"✅ Final  : {final_cmd}")
    print(f"🔍 Valid  : {'YES' if valid else 'NO'}\n")



🧪 MEDIUM COMMAND TESTS (WITH RULES)

📝 Prompt : Scan all ports on 192.168.1.20
🤖 Raw    : nmap -p- 192.168.1.20
✅ Final  : nmap -p- 192.168.1.20
🔍 Valid  : YES

📝 Prompt : Scan all ports and detect services on 192.168.1.20
🤖 Raw    : nmap -p- -sV 192.168.1.20
✅ Final  : nmap -p- -sV 192.168.1.20
🔍 Valid  : YES

📝 Prompt : Scan all TCP ports using SYN scan on 192.168.1.20
🤖 Raw    : nmap -p- -sS 192.168.1.20
✅ Final  : nmap -p- -sS 192.168.1.20
🔍 Valid  : YES

📝 Prompt : Scan top ports with version detection on 192.168.1.50
🤖 Raw    : nmap -p- -sV 192.168.1.50
🛠 Fixed  : nmap  -sV 192.168.1.50
🔍 Valid  : YES

📝 Prompt : Run default scripts on 192.168.1.10
🤖 Raw    : nmap -p 80 --script default 192.168.1.10
✅ Final  : nmap -p 80 --script default 192.168.1.10
🔍 Valid  : YES

📝 Prompt : Run default and vuln scripts on 192.168.1.10
🤖 Raw    : nmap -p- 192.168.1.10
🛠 Fixed  : nmap --script default,vuln -p- 192.168.1.10
🔍 Valid  : YES

📝 Prompt : Scan a subnet for live hosts using ping scan 